In [ ]:
# EXTRAÇÃO DE COR

from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
from PIL import Image
import pandas as pd

def extract_dominant_color(image_url):
    try:
        # Abrir a URL diretamente usando urllib
        image = Image.open(urllib.request.urlopen(image_url))

        # Converte a imagem para o formato RGB
        image = image.convert("RGB")

        # Converte a imagem para um array NumPy
        image_array = np.array(image)

        # Obter as dimensões da imagem
        w, h, d = tuple(image_array.shape)

        # Remodelar para um array 2D de pixels
        pixels = np.reshape(image_array, (w * h, d))

        # COR PREDOMINANTE
        n_colors = 1

        # Criar o modelo KMeans e aplicar aos pixels
        model = KMeans(n_clusters=n_colors, random_state=42).fit(pixels)

        # Obter a paleta de cores (centros dos clusters)
        colour_palette = np.uint8(model.cluster_centers_)

        # Obter o código hex da cor predominante
        dominant_color = colour_palette[0]
        hex_color = '#{:02x}{:02x}{:02x}'.format(dominant_color[0], dominant_color[1], dominant_color[2])

        return hex_color
    except Exception as e:
        print(f"Erro ao processar a imagem: {e}")
        return None

In [ ]:
import pandas as pd
import numpy as np

# LE OS ARQUIVOS JÁ PROCESSADOS ANTERIORMENTE

cores_css=pd.read_csv('cores_basicas_css.csv') # CORES BASICAS NOMEADAS DO CSS
todos_albums=pd.read_csv('completo_todas_as_musicas_processadas.csv') # DF COM ALBUMS PROCESSADOS
todos_albums=todos_albums.dropna() # REMOVE VALORES VAZIOS

# Função para converter "rgb(r, g, b)" em uma tupla (r, g, b)
def parse_rgb(rgb_string):
    rgb_string = rgb_string.strip("rgb()")
    return tuple(map(int, rgb_string.split(',')))

# Adicionar a coluna `rgb_tuple` ao cores_df com base em `cor_rgb`
cores_css['rgb_tuple'] = cores_css['cor_rgb'].apply(parse_rgb)

# Função para converter HEX para RGB
def hex_to_rgb(hex_color):
    if not isinstance(hex_color, str) or len(hex_color) < 7 or not hex_color.startswith('#'):
        raise ValueError(f"Cor inválida: {hex_color}")
    hex_color = hex_color.lstrip('#')
    return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))

# Função para encontrar a cor mais próxima
def cor_mais_proxima(cor_predominante):
    try:
        rgb_pred = hex_to_rgb(cor_predominante)
    except ValueError:
        return None  # Retornar None para cores inválidas
    
    menor_distancia = float('inf')
    cor_proxima = None

    for _, row in cores_css.iterrows():
        rgb = row['rgb_tuple']
        distancia = np.sqrt(sum((rgb_pred[i] - rgb[i]) ** 2 for i in range(3)))
        if distancia < menor_distancia:
            menor_distancia = distancia
            cor_proxima = row['nome']
    
    return cor_proxima

# Aplicar a lógica ao segundo DataFrame
todos_albums['cor_mais_proxima'] = todos_albums['cor_predominante'].apply(cor_mais_proxima)

In [ ]:
# CRUZA PARA ACHAR O #HEX DA COR MAIS PRÓXIMA
cores_css['cor_mais_proxima']=cores_css['nome']

todos_albums_final=pd.merge(todos_albums,cores_css,on='cor_mais_proxima',how='left')
todos_albums_final.columns=['id_artista', 'nome_artista', 'id_album', 'nome_album', 'album_type',
       'release_date', 'album_group', 'capa_album', 'artistas_album',
       'cor_predominante', 'cor_mais_proxima', 'nome_cor_proxima', 'categoria', 'hex_cor_proxima',
       'cor_rgb_proxima', 'rgb_tuple_proxima']

# COLOCA ANO

todos_albums_final['ano']=todos_albums_final['release_date'].str[:4]
todos_albums_final.to_csv('arquivo_final_analise.csv',index=False)